# 2D List of facts

Given a list of facts, e.g., `F = [(1, 'N', 2), (2, 'N', 3), (3, 'N', 1)]` about points on a 2D plane, determine if these points can or cannot be ploted.

In [23]:
from dataclasses import dataclass, field
from typing import Mapping, List, Tuple, Set
from collections import defaultdict

NodeMap = Mapping[int, List[int]]
Fact = Tuple[int, str, int]

@dataclass
class Node:
    value: int
    next: List['Node'] = field(default_factory = list)

def can_plot_facts(facts: List[Fact]) -> bool:
    x_vertices, y_vertices = defaultdict(list), defaultdict(list)
    for (A, edge, B) in facts:
        match edge:
            case 'N':
                y_vertices[A].append(B)
            case 'S':
                y_vertices[B].append(A)
            case 'E':
                x_vertices[A].append(B)
            case 'W':
                x_vertices[B].append(A)
            case _:
                raise NotImplemented("Direction unkown.")
    return not has_cycle(x_vertices) and not has_cycle(y_vertices)

def build_graph(vertices: NodeMap) -> bool:
    root = None
    node_map = {}
    for (value, next_values) in vertices.items():
        if value not in node_map:
            current_node = Node(value)
            node_map[value] = current_node
        else:
            current_node = node_map[value]
        if root is None:
            root = current_node
        for next_value in next_values:
            if next_value not in node_map:
                node_map[next_value] = Node(next_value)
            current_node.next.append(node_map[next_value])
    return root

def has_cycle(vertices: NodeMap) -> bool:
    def has_cycle_aux(node: Node, visited: Set = set()) -> bool:
        cycle = False
        if node:
            if node.value in visited:
                return True
            visited.add(node.value)
            for next_node in node.next:
                cycle = has_cycle_aux(next_node, visited)
        return cycle
    
    root = build_graph(vertices)
    return has_cycle_aux(root)
        


In [35]:
import unittest

class TestCanPlotFacts(unittest.TestCase):
  def test_can_plot_facts(self):
    facts_examples = [
        {   
            "case": "North Cycle",
            "facts": [
                (1, 'N', 2),
                (2, 'N', 3),
                (3, 'N', 1)
            ],
            "expected": False
        },
        {
            "case": "No cycle",
            "facts": [
                (1, 'N', 2),
                (2, 'N', 3),
            ],
            "expected": True
        },    
        {
            "case": "Empty facts",
            "facts": [
            ],
            "expected": True
        },    
        {
            "case": "West Cycle",
            "facts": [
                (1, 'N', 2),
                (2, 'N', 3),
                (3, 'W', 1),
                (4, 'W', 3),
                (1, 'W', 4)
            ],
            "expected": False
        },                    
    ]
    for example in facts_examples:
        with self.subTest(example["case"]):
            actual = can_plot_facts(example["facts"])
            self.assertEqual(actual, example["expected"])

unittest.main(argv=[''], verbosity=2, exit=False)

test_can_plot_facts (__main__.TestCanPlotFacts) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
